In [48]:
pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
from google.colab import files
uploaded = files.upload()

Saving data.txt to data.txt


In [84]:
%%file q1.py 
from mrjob.job import MRJob

class q1(MRJob):
    def mapper(self, _, line):
       name,date,*songs=line.split(' ')
       if(len(songs)>5):
          yield(name,date),len(songs)    
        
if __name__ == '__main__':
    q1.run()

Writing q1.py


In [85]:
! python q1.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q1.root.20230322.182154.237094
Running step 1 of 1...
job output is in /tmp/q1.root.20230322.182154.237094/output
Streaming final output from /tmp/q1.root.20230322.182154.237094/output...
["Ali", "12-3-20"]	6
Removing temp directory /tmp/q1.root.20230322.182154.237094...


**Q2**

In [90]:
%%file q2.py 
import sys
from mrjob.job import MRJob

class q2(MRJob):
    def mapper(self, _, line):
        username, date, *songs = line.strip().split(' ')
        yield (username,date),len(songs)
        
if __name__ == '__main__':
    q2.run()

Overwriting q2.py


In [91]:
!python q2.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q2.root.20230322.182314.862985
Running step 1 of 1...
job output is in /tmp/q2.root.20230322.182314.862985/output
Streaming final output from /tmp/q2.root.20230322.182314.862985/output...
["Fahad", "14-04-22"]	4
["Ali", "20-4-22"]	2
["Sara", "20-4-22"]	3
["Amina", "20-4-22"]	1
["Ali", "12-3-20"]	6
Removing temp directory /tmp/q2.root.20230322.182314.862985...


**Q3**

In [94]:
%%file q3.py 
from mrjob.job import MRJob
from mrjob.job import MRStep

class q3(MRJob):
    def steps(self):
        return[MRStep(mapper=self.mapper,combiner=self.combiner,reducer=self.reducer)]
    def mapper(self,_,line):
        username,date,*songs=line.strip().split(' ')
        yield username,len(songs)
    def combiner(self,username,count):
        total=sum(count)
        yield username,total    
    def reducer(self,username,count):
        total=sum(count)
        yield username,total

if __name__ == '__main__':
    q3.run()

Overwriting q3.py


In [95]:
!python q3.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q3.root.20230322.182349.410118
Running step 1 of 1...
job output is in /tmp/q3.root.20230322.182349.410118/output
Streaming final output from /tmp/q3.root.20230322.182349.410118/output...
"Amina"	1
"Fahad"	4
"Sara"	3
"Ali"	8
Removing temp directory /tmp/q3.root.20230322.182349.410118...


**Q4**

In [97]:
%%file q4.py 
from mrjob.job import MRJob
from mrjob.job import MRStep

class q4(MRJob):
    
    def mapper(self, _, line):
        username, date, *songs = line.strip().split(' ')
        yield None, len(songs)

    def combiner(self, _, song_counts):
        total_songs = 0
        total_users = 0
        for count in song_counts:
            total_songs += count
            total_users += 1
        average = total_songs / total_users
        yield  None,average
        
        
    def reducer(self, _, song_counts):
        total_songs = 0
        total_users = 0
        for count in song_counts:
            total_songs += count
            total_users += 1
        average = total_songs / total_users
        yield  None,average    
        
    def steps(self):
        return [MRStep(mapper=self.mapper,combiner=self.combiner, reducer=self.reducer)]
        
if __name__ == '__main__':
    q4.run()

Overwriting q4.py


In [98]:
!python q4.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q4.root.20230322.182452.965968
Running step 1 of 1...
job output is in /tmp/q4.root.20230322.182452.965968/output
Streaming final output from /tmp/q4.root.20230322.182452.965968/output...
null	3.6666666666666665
Removing temp directory /tmp/q4.root.20230322.182452.965968...


**Q5**

In [100]:
%%file q5.py
from mrjob.job import MRJob
from mrjob.job import MRStep

class q5(MRJob):
    def mapper(self, _, line):
        fields = line.strip().split(' ')
        username, date, *songs = fields
        for song in songs:
            yield song, 1

    def combiner(self, song, counts):
        yield song, sum(counts)
    def reducer(self, song, counts):
        yield song, sum(counts)
    def step(self):
        return [MRStep(mapper=self.mapper,combiner=self.combiner,reducer=self.reducer)]
        
if __name__ == '__main__':
    q5.run()

Overwriting q5.py


In [101]:
!python q5.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q5.root.20230322.182549.006164
Running step 1 of 1...
job output is in /tmp/q5.root.20230322.182549.006164/output
Streaming final output from /tmp/q5.root.20230322.182549.006164/output...
"S3"	2
"S6"	1
"S7"	3
"S9"	2
"S1"	5
"S2"	3
Removing temp directory /tmp/q5.root.20230322.182549.006164...


**Q6**

In [104]:
%%file q6.py
from mrjob.job import MRJob
from mrjob.job import MRStep

class q6(MRJob):

    def mapper(self, _, line):
        fields = line.strip().split(' ')
        username, date, *songs = fields
        yield date, (username, len(songs))

    def combiner(self, date, values):
        max_user, max_songs = None, 0
        for user, song_count in values:
            if song_count > max_songs:
                max_songs = song_count
                max_user = user
        yield date, (max_user, max_songs)

    def reducer(self, date, values):
        max_user, max_songs = None, 0
        for user, song_count in values:
            if song_count > max_songs:
                max_songs = song_count
                max_user = user
        yield date, (max_user, max_songs)
        
    def step(self):
        return [MRStep(mapper=self.mapper,combiner=self.combiner,reducer=self.reducer)]

if __name__ == "__main__":
    q6.run()
        

Overwriting q6.py


In [105]:
!python q6.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q6.root.20230322.182639.475019
Running step 1 of 1...
job output is in /tmp/q6.root.20230322.182639.475019/output
Streaming final output from /tmp/q6.root.20230322.182639.475019/output...
"20-4-22"	["Sara", 3]
"12-3-20"	["Ali", 6]
"14-04-22"	["Fahad", 4]
Removing temp directory /tmp/q6.root.20230322.182639.475019...


**Q7**

In [ ]:
!python q7.py data.txt

In [110]:
%%file q8.py
from mrjob.job import MRJob
import statistics
class q8(MRJob):
    
    def mapper(self, _, line):
       name,date,*songs=line.split(' ')
       yield date,(name,len(songs))
         
    def combiner(self, key, values):
        yield key, max(values)  

    def reducer(self, key, values):
        yield key, max(values) 

if __name__ == '__main__':
    q8.run()

Overwriting q8.py


In [111]:
!python q8.py data.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/q8.root.20230322.182915.831148
Running step 1 of 1...
job output is in /tmp/q8.root.20230322.182915.831148/output
Streaming final output from /tmp/q8.root.20230322.182915.831148/output...
"20-4-22"	["Sara", 3]
"12-3-20"	["Ali", 6]
"14-04-22"	["Fahad", 4]
Removing temp directory /tmp/q8.root.20230322.182915.831148...
